In [1]:
print("hi")

import os
import pandas as pd

import numpy as np

import csv

from robust_evaluation_tools.robust_harmonization import fit, apply
from robust_evaluation_tools.robust_utils import get_info, get_bundles, get_complete_combination, get_diseases
import subprocess

raw_directory = 'DONNES/RAW_PER_DISEASE'

method= "classic"

hi


In [ ]:
mat =[]
fields = ["Name","NB_Suject","NB_HC","NB_Sick", "Disease"]

metric = "md"
CAMCAN = f"./DONNES/CamCAN.{metric}.raw.csv.gz"
disease_list = []
for disease_folder in sorted(os.listdir(raw_directory)):
    disease_list.append(disease_folder)
    disease_path = os.path.join(raw_directory, disease_folder)
    print(disease_folder)
    if os.path.isdir(disease_path):
        for filename in sorted(os.listdir(os.path.join(disease_path, metric))):
            f = os.path.join(disease_path, metric, filename)
            # checking if it is a file
            if os.path.isfile(f):
                #print(f)
                infos = [filename.split('.')[0]] 
                infos = infos + get_info(f) + [disease_folder]
                #fit(f, CAMCAN, metric)
                #apply(f, metric)
                mat.append(infos)
                #print("---------------------------")
disease_sites_df  = pd.DataFrame(mat, columns=fields) 
filtered_disease_sites_df = disease_sites_df [(disease_sites_df ['NB_HC'] >= 8) & (disease_sites_df ['NB_Sick'] >= 5)]
disease_list.remove('HC')


In [ ]:
filtered_disease_sites_df

In [2]:
metrics = [
    "ad",
    "adt",
    "afd",
    "fa",
    "fat",
    "fw",
    "md",
    "mdt",
    "rd",
    "rdt"
]

In [ ]:
harmonizartion_method = "classic"
robust = 'No'
rwp = False
hc = True
models_directory = "MODELS_F"
clean_directory = "DONNES_F/HARMONIZED_SITES"
os.makedirs(clean_directory, exist_ok=True)
for disease in disease_list:
    os.makedirs(f"DONNES_F/COMPILATIONS/{disease}", exist_ok=True)
    all = pd.DataFrame()
    for metric in metrics:
        combination = pd.DataFrame()
        CAMCAN = f"./DONNES_F/CamCAN/CamCAN.{metric}.raw.csv.gz"
        for site in filtered_disease_sites_df[filtered_disease_sites_df['Disease'] == disease]['Name'].values:
            f = os.path.join(raw_directory,disease, metric,  site + "." + metric + ".raw.csv.gz")
            # checking if it is a file
            if os.path.isfile(f):
                print(f)
                model_file = fit(f, CAMCAN, metric, harmonizartion_method, robust, rwp, models_directory, hc,)
                clean_site_file = apply(f, model_file, metric, harmonizartion_method, robust, rwp, clean_directory)
                df = pd.read_csv(clean_site_file)
                combination = pd.concat([combination, df], ignore_index=True)
                all = pd.concat([all, df], ignore_index=True)
                print("---------------------------")
    
        combination.to_csv(f"DONNES_F/COMPILATIONS/{disease}/{disease}_combination.{metric}.csv.gz", index=False, compression='gzip')
    all.to_csv(f"DONNES_F/COMPILATIONS/{disease}/{disease}_all_metrics.csv.gz", index=False, compression='gzip')

In [3]:
raw_directory = os.path.join('DONNES_F', 'COMPILATIONS')
for d in get_diseases(False):
        disease = d
        print('--------------------', disease, '-----------------------')
        combination = get_complete_combination(os.path.join(raw_directory, disease), f'{disease}_combination*.csv.gz', True, False)
        combination['metric_bundle'] = combination['metric'] + "_" + combination['bundle']
        combination.to_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'), index=False)
        # combination.to_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics.csv.gz'), index=False)

-------------------- AD -----------------------
-------------------- ADHD -----------------------
-------------------- BIP -----------------------
-------------------- MCI -----------------------
-------------------- SCHZ -----------------------
-------------------- TBI -----------------------


In [ ]:
# print("Visualize")
# directory = "RESULTS/COMPILATION_VIZUALIZE"
# for disease in disease_list:
#     for metric in metrics:
#         # Define the file
#         filename = f'DONNES/COMPILATIONS/{disease}/{disease}_combination.{metric}.csv.gz'
#         CAMCAN = f"./DONNES/CamCAN/CamCAN.{metric}.raw.csv.gz"
#         # Check if it is a file
#         if os.path.isfile(filename):
#             print(filename)
#             dir = f"COMPILATION_VIZUALIZE/{disease}/{metric}"
#             cmd = (
#                 "combat_visualize_data.py"
#                 + " "
#                 + filename
#                 + " "
#                 + CAMCAN
#                 + " --out_dir "
#                 + dir
#                 + " --bundles all"
#             )
#             subprocess.call(cmd, shell=True)
#             print("---------------------------")